# Plan for Python work

1) My _suggestion_ is to proceed as before and work through the notebook , switching every 5 or so minutes, discussing in each cell what the lines mean.  You may do something differently if you both agree. 

2) I provide a number of commands below (and prompts) , some of the things you need to figure out yourself and some of the tasks I have completed for you 

3) Make sure you understand how much time you have (ask me if you don't know) and plan accordingly. There is a lot of infromation in here!

4) Plenty of suggestions at the bottom for more things to try - you should take a look and make sure you can do all of these things... 

In [48]:
#A bunch of libraries and packages 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib2

from sklearn import linear_model
from pandas.tools.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
%matplotlib inline

In [2]:
# UCI ML database - energy efficiency
# Database of many ML data available here: https://archive.ics.uci.edu/ml/

#what the heck is this doing? 
socket = urllib2.urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx')
UCI_energy=pd.read_excel(socket)

# definition of dataframes 
# X1	Relative Compactness 
# X2	Surface Area 
# X3	Wall Area 
# X4	Roof Area 
# X5	Overall Height 
# X6	Orientation 
# X7	Glazing Area 
# X8	Glazing Area Distribution 
# y1	Heating Load 
# y2	Cooling Load

In [26]:
# This is a naive validation set approach. Please understand and briefly discuss this is just for teaching
# What would you do in the real world based on our bootstrap/resampling lessons? 
train,test=train_test_split(UCI_energy,test_size=0.05,random_state=1010)

# have you had a look at the data yet? quickly do so before moving on...

# Part 1: Multiple linear regression on X1-X8 predicting Y1 

In [ ]:
# train linear model 
MLR=linear_model.LinearRegression()
MLR.fit(train[['X1','X2','X3','X4','X5','X6','X7','X8']],train.Y1)

# make predictions on test and train set 
trainpred=MLR.predict(train[['X1','X2','X3','X4','X5','X6','X7','X8']])
testpred=MLR.predict(test[['X1','X2','X3','X4','X5','X6','X7','X8']])

#make parity plot 
plt.figure(figsize=(4,4))
plt.xlim([0,50]);
plt.ylim([0,50]);
plt.scatter(train.Y1,trainpred)
plt.scatter(test.Y1,testpred,color='r')
plt.plot([0,50],[0,50],lw=4,color='black')

#calculate the test and train error
print("Train error",mean_squared_error(train.Y1,trainpred))
print("Test error",mean_squared_error(test.Y1,testpred))

# Part 2: Ridge Regression (same data as Part 1)

* The Ridge coefficients minimize $RSS + \lambda \sum_{j=1}^{p}\beta_j^2$
    * There is an additional **penalty** in error for having nonzero coefficients!
* Note: Eq 6.5 in ISLR shows the tuning parameter as $\lambda$, it is $\alpha$ in SKLearn
* Goal here: train models as a function of the regularization parameter 
* The X's should be normalized as in Eq 6.6, there is a normalization feature, but we will do it manually using $x_{ij}=\frac{x_{ij}}{s_j}$
    * I suggest on your own you test out what normalization in Ridge does
    * Some methods in sklearn also do automatic selection of shrinkage coefficient! Cool! 

In [62]:
#normalized data for Ridge / LASSO 
train_normalized=train/train.std()
test_normalized=test/test.std()


## 2-1 Example of single instance of RR


In [85]:
heat_ridge=Ridge()
a=1e0
heat_ridge.set_params(alpha=a)
heat_ridge.fit(train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']],train_normalized.Y1)


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [86]:
print mean_squared_error(train_normalized.Y1,heat_ridge.predict(
        train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']]))

print mean_squared_error(test_normalized.Y1,heat_ridge.predict(
        test_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']]))

0.0818431981701
1.24921520518


## 2-2 Example of searching the $\alpha$ space in RR

In [ ]:
# RR vs lambda (based on sklearn tutorial)
coefs = []
trainerror = []
testerror = []

# do you know what is happening here? 
lambdas = np.logspace(-6,6,200)
model=Ridge()

# loop over lambda values (strength of regularization)
for l in lambdas:
    model.set_params(alpha=l)
    model.fit(train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']],train_normalized.Y1)
    coefs.append(model.coef_)
    trainerror.append(mean_squared_error(train_normalized.Y1,model.predict(
        train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']])))
    testerror.append(mean_squared_error(test_normalized.Y1,model.predict(
        test_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']])))

    

In [ ]:
# what is being plotted here? 
plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(lambdas,coefs)
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('coefs')
plt.title('RR coefs vs $\lambda$')
plt.subplot(122)
plt.plot(lambdas,trainerror,label='train error')
plt.plot(lambdas,testerror,label='test error')
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('error')
plt.legend(loc=1)
plt.title('error vs $\lambda$')


### RR questions 

1) Explain to each other what is happening in these two plots   
2) Why does the blue curve have a minimum at the smallest $\lambda4 value? 

# Part 3: LASSO regression  (same data as Part 1)

* The lasso improves over ridge by also providing a variable selection tool!
* The lasso minimizer is $RSS + \lambda \sum_{j=1}^{p}\lvert\beta_j\rvert$

In [117]:
# also based on sklearn tutorials
# what the hell is happening in this cell?
coefs = []
trainerror = []
testerror = []

lambdas = np.logspace(-6,6,200)
model=linear_model.Lasso()

# loop over lambda values (strength of regularization)
for l in lambdas:
    model.set_params(alpha=l,max_iter=1e6)
    model.fit(train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']],train_normalized.Y1)
    coefs.append(model.coef_)
    trainerror.append(mean_squared_error(train_normalized.Y1,model.predict(
        train_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']])))
    testerror.append(mean_squared_error(test_normalized.Y1,model.predict(
        test_normalized[['X1','X2','X3','X4','X5','X6','X7','X8']])))



In [ ]:
plt.figure(figsize=(10,3))
#plt.locator_params(nbins=5)
plt.subplot(121)
plt.plot(lambdas,coefs)
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('coefs')
plt.title('RR coefs vs $\lambda$')
#plt.xlim(1e-4,1e0)

plt.subplot(122)
plt.plot(lambdas,trainerror,label='train error')
plt.plot(lambdas,testerror,label='test error')
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('error')
#plt.xlim(1e-4,1e0)
#plt.ylim(0,0.5)
plt.legend(loc=1)
plt.title('error vs $\lambda$')

### Other things to consider if you have more time 

* Note we did not scale the features in the MLR, try it out and verify the final error doesnt' change!
* Make sure you undersand how to make _predictions_ with supervised learning models that are trained on scaled/normalized data
* Plot the residuals and verify if errors are distributed normally
* Make a parity plot including the predictions from ridge and LASSO 
* Compare errors between all three 
* Explore the effect of training/testing split 
* Look at the shrinkage/regularization situation when predicting Y2 vs Y1... 
